<a href="https://colab.research.google.com/github/risker93/Hello_World/blob/main/daily/2021_07_01_NaverMovieReview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")


('ratings_test.txt', <http.client.HTTPMessage at 0x7f7d597469d0>)

In [ ]:
train_data=pd.read_table('ratings_train.txt')

In [ ]:
def read_data(filename):
  with open(filename,'r') as f:
    data = [line.split('\t') for line in f.read().splitlines()]

    data = data[1:] # txt file의 header 없이 사용하겠다.
  return data

train_data=read_data('ratings_train.txt')
test_data = read_data('ratings_test.txt')

In [ ]:
len(train_data)

150000

In [ ]:
len(train_data[0])

3

In [ ]:
pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 450kB 41.5MB/s 
     |████████████████████████████████| 92kB 10.4MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import konlpy
from konlpy.tag import Okt

okt = Okt()
okt.pos(u'이 밤 그날의 반딧불을 당신의 창 가까이 보낼게요')

[('이', 'Noun'),
 ('밤', 'Noun'),
 ('그날', 'Noun'),
 ('의', 'Josa'),
 ('반딧불', 'Noun'),
 ('을', 'Josa'),
 ('당신', 'Noun'),
 ('의', 'Josa'),
 ('창', 'Noun'),
 ('가까이', 'Noun'),
 ('보낼게요', 'Verb')]

In [ ]:
import json # python dic()
import os
from pprint import pprint

def tokenize(doc):
  return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

In [ ]:
if os.path.isfile('train_docs.json'):
  with open('train_docs.json') as f:
    train_docs = json.load(f)
  with open('test_docs.json') as f:
    test_docs = json.load(f)

else:
  train_docs = [(tokenize(row[1]), row[2]) for row in train_data]
  test_docs = [(tokenize(row[1]), row[2]) for row in test_data]

  with open('train_docs.json', 'w', encoding='utf-8') as make_file:
    json.dump(train_docs, make_file, ensure_ascii=False, indent='\t')
  with open('test_docs.json', 'w', encoding='utf-8') as make_file:
    json.dump(test_docs, make_file, ensure_ascii=False, indent='\t')

pprint(train_docs[0])

(['아/Exclamation',
  '더빙/Noun',
  '../Punctuation',
  '진짜/Noun',
  '짜증나다/Adjective',
  '목소리/Noun'],
 '0')


In [ ]:
tokens = [t for d in train_docs for t in d[0]]
len(tokens)

2159921

In [ ]:
import nltk

text = nltk.Text(tokens, name='NMSC')

print(len(text.tokens)) # 전체 토큰 개수

2159921


In [ ]:
print(len(set(text.tokens)))

49895


In [ ]:
pprint(text.vocab().most_common(10))


[('./Punctuation', 67778),
 ('영화/Noun', 50818),
 ('하다/Verb', 41209),
 ('이/Josa', 38540),
 ('보다/Verb', 38538),
 ('의/Josa', 30188),
 ('../Punctuation', 29055),
 ('가/Josa', 26627),
 ('에/Josa', 26468),
 ('을/Josa', 23118)]


In [ ]:
selected_words = [f[0] for f in text.vocab().most_common(100)]

def term_frequency(doc):
  return [doc.count(word) for word in selected_words]
  

In [ ]:
train_x = [term_frequency(d) for d, _ in train_docs]
test_x = [term_frequency(d) for d, _ in test_docs]

train_y = [c for _, c in train_docs]
test_y = [c for _, c in train_docs]

In [ ]:
import numpy as np 

X_train = np.asarray(train_x).astype('float32')
X_test = np.asarray(test_x).astype('float32')

y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')


In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(150000, 100)
(50000, 100)
(150000,)
(150000,)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(100,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 0.001),
              loss=losses.binary_crossentropy,
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, batch_size=512)
model.evaluate(x_test, y_test)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


NameError: ignored